In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('final.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          13495 non-null  int64  
 1   order_id         13495 non-null  int64  
 2   Date             13495 non-null  object 
 3   Amount           13495 non-null  float64
 4   Merchant_id      13495 non-null  int64  
 5   Purchase_status  13495 non-null  int64  
 6   Merchant_name    13495 non-null  object 
 7   Rate_pct         13495 non-null  int64  
 8   Revenue          13495 non-null  float64
 9   First_tran_date  13495 non-null  object 
 10  Location         13495 non-null  object 
 11  Age              13495 non-null  object 
 12  Gender           13495 non-null  object 
 13  Type_user        13495 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 1.4+ MB


In [5]:
df['Date'] = pd.to_datetime(df['Date'])
df['First_tran_date'] = pd.to_datetime(df['First_tran_date'])

In [6]:
cat_cols = [
    'Merchant_name',
    'Location',
    'Age',
    'Gender',
    'Type_user'
]

for col in cat_cols:
    df[col] = df[col].astype('category')

In [7]:
df['Purchase_status'] = df['Purchase_status'].astype('int8')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13495 non-null  int64         
 1   order_id         13495 non-null  int64         
 2   Date             13495 non-null  datetime64[ns]
 3   Amount           13495 non-null  float64       
 4   Merchant_id      13495 non-null  int64         
 5   Purchase_status  13495 non-null  int8          
 6   Merchant_name    13495 non-null  category      
 7   Rate_pct         13495 non-null  int64         
 8   Revenue          13495 non-null  float64       
 9   First_tran_date  13495 non-null  datetime64[ns]
 10  Location         13495 non-null  category      
 11  Age              13495 non-null  category      
 12  Gender           13495 non-null  category      
 13  Type_user        13495 non-null  category      
dtypes: category(5), datetime64[ns](2), flo

In [10]:
df['month'] = df['Date'].dt.to_period('M').astype(str)
df['year'] = df['Date'].dt.year

In [11]:
mart_growth = (
    df.groupby('month')
      .agg(
          active_users = ('user_id','nunique'),
          new_users = ('user_id',
                       lambda x: x[df.loc[x.index,'Type_user']=='New'].nunique())
      )
      .reset_index()
)

mart_growth['current_users'] = (
    mart_growth['active_users'] - mart_growth['new_users']
)

mart_growth['pct_new_users'] = (
    mart_growth['new_users'] / mart_growth['active_users'])

In [12]:
mart_growth

,month,active_users,new_users,current_users,pct_new_users
0,2020-01,1000,85,915,0.085000
1,2020-02,969,91,878,0.093911
2,2020-03,1098,113,985,0.102914
3,2020-04,1027,81,946,0.078870
4,2020-05,1104,72,1032,0.065217
5,2020-06,1116,62,1054,0.055556
6,2020-07,1149,72,1077,0.062663
7,2020-08,1153,84,1069,0.072853
8,2020-09,1189,82,1107,0.068966
9,2020-10,1209,79,1130,0.065343


In [ ]:
mart_revenue = (
    df.groupby('month')
    .agg(
        total_txn = ('order_id','count'),
          total_amount = ('Amount','sum'),
          revenue = ('Revenue','sum'),
          active_users = ('user_id','nunique')
      )
      .reset_index()
)

mart_revenue['AOV'] = mart_revenue['total_amount'] / mart_revenue['total_txn']
mart_revenue['revenue_per_user'] = mart_revenue['revenue'] / mart_revenue['active_users']
mart_revenue['take_rate'] = mart_revenue['revenue'] / mart_revenue['total_amount']

In [14]:
mart_revenue

,month,total_txn,total_amount,revenue,active_users,AOV,revenue_per_user,take_rate
0,2020-01,1000,53814234.0,140982702.0,1000,53814.234000,140982.702000,2.619803
1,2020-02,972,52680000.0,137850000.0,969,54197.530864,142260.061920,2.616743
2,2020-03,1098,58200000.0,158400000.0,1098,53005.464481,144262.295082,2.721649
3,2020-04,1027,54150000.0,148830000.0,1027,52726.387537,144917.234664,2.748476
4,2020-05,1104,52950000.0,146370000.0,1104,47961.956522,132581.521739,2.764306
5,2020-06,1118,58250000.0,161720000.0,1116,52101.967800,144910.394265,2.776309
6,2020-07,1153,58940000.0,158190000.0,1149,51118.820468,137676.240209,2.683916
7,2020-08,1153,59950000.0,161870000.0,1153,51994.796184,140390.286210,2.700083
8,2020-09,1189,63400000.0,170220000.0,1189,53322.119428,143162.321278,2.684858
9,2020-10,1211,61690000.0,169090000.0,1209,50941.370768,139859.387924,2.740963


In [15]:
mart_merchant = (
    df.groupby(['month','Merchant_name'])
      .agg(
          users = ('user_id','nunique'),
          txn = ('order_id','count'),
          amount = ('Amount','sum'),
          revenue = ('Revenue','sum')
      )
      .reset_index()
)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16524\1533046893.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['month','Merchant_name'])


In [16]:
mart_behavior = (
    df.groupby(['month','Type_user'])
      .agg(
          users = ('user_id','nunique'),
          txn = ('order_id','count'),
          revenue = ('Revenue','sum')
      )
      .reset_index()
)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16524\1503217853.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['month','Type_user'])


In [17]:
df['first_month'] = df['First_tran_date'].dt.to_period('M').astype(str)

mart_cohort = (
    df.groupby(['first_month','month'])
      .agg(users=('user_id','nunique'))
      .reset_index()
)

In [22]:
import os

output_dir = "C://Users//ASUS//Documents//Project//Momo case study//Data//processed//mart"
os.makedirs(output_dir, exist_ok=True)

In [24]:
mart_behavior.to_csv(f"{output_dir}/mart_behavior.csv", encoding = 'utf-8-sig', index = False)
mart_revenue.to_csv(f"{output_dir}/mart_revenue.csv", encoding = 'utf-8-sig', index = False)
mart_growth.to_csv(f"{output_dir}/mart_growth.csv", encoding = 'utf-8-sig', index = False)
mart_cohort.to_csv(f"{output_dir}/mart_cohort.csv", encoding = 'utf-8-sig', index = False)
mart_merchant.to_csv(f"{output_dir}/mart_merchant.csv", encoding = 'utf-8-sig', index = False)